In [8]:
from sklearn.pipeline import FunctionTransformer, Pipeline, make_pipeline
from wind_power.dataset import InfluxDBClientWrapper, get_power_and_wind_data
from darts import TimeSeries
from wind_power.features import robust_timeseries_imputer, DirectionEncodingTransformer
from darts.models import VARIMA



with InfluxDBClientWrapper() as client:
    power_df, wind_df, today = get_power_and_wind_data(
        client, days=250, date="2025-02-10"
    )

power_df = power_df[["Total"]]
wind_df.drop(columns=["Lead_hours", "Source_time"], inplace=True)

joined_dfs = power_df.join(wind_df, how="right")

pipeline = Pipeline(
    [
        ("impute", FunctionTransformer(robust_timeseries_imputer)),
        ("encode", DirectionEncodingTransformer("vector")),
        (
            "timeseries",
            FunctionTransformer(lambda df: TimeSeries.from_dataframe(df)),
        ),
        ("Model", VARIMA()),
    ]
)
pipeline.fit(joined_dfs)


/home/macja/repositories/a1_lsda2025/wind_power/features.py:94: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_imputed = df.interpolate(method="time")
/home/macja/repositories/a1_lsda2025/wind_power/features.py:97: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_imputed = df_imputed.fillna(method="ffill").fillna(method="bfill")  # type: ignore
/home/macja/.pyenv/versions/a1_lsda2025/lib/python3.11/site-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future the

Pipeline(steps=[('impute',
                 FunctionTransformer(func=<function robust_timeseries_imputer at 0x7f6cb85d60c0>)),
                ('encode',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('direction',
                                                  FunctionTransformer(func=<function direction_to_vector at 0x7f6cb96959e0>),
                                                  ['Direction'])],
                                   verbose_feature_names_out=False)),
                ('timeseries',
                 FunctionTransformer(func=<function <lambda> at 0x7f6c95333920>)),
                ('Model',
                 VARIMA(p=1, d=0, q=0, trend=None, add_encoders=None))])

In [ ]:
VARIMA().predict()

In [ ]:
VARIMA().predict

In [ ]:
pipeline.named_steps['Model'].predit()

<TimeSeries (DataArray) (time: 30, component: 4, sample: 1)> Size: 960B
array([[[-0.81835044],
        [ 0.28259768],
        [ 7.74912582],
        [ 4.20918838]],

       [[-0.72504456],
        [ 0.19849614],
        [ 8.37633644],
        [ 4.38391116]],

       [[-0.64283811],
        [ 0.12834168],
        [ 8.7929555 ],
        [ 4.53698217]],

       [[-0.57056767],
        [ 0.07019814],
        [ 9.05823954],
        [ 4.66364391]],

...

       [[-0.06171424],
        [-0.13922161],
        [ 6.39476396],
        [ 3.6211306 ]],

       [[-0.05739733],
        [-0.13633376],
        [ 6.23340992],
        [ 3.53278297]],

       [[-0.05348585],
        [-0.13338988],
        [ 6.07467635],
        [ 3.44545849]],

       [[-0.04993609],
        [-0.13041155],
        [ 5.91869801],
        [ 3.35929634]]])
Coordinates:
  * time       (time) datetime64[ns] 240B 2025-02-10T03:00:00 ... 2025-02-13T...
  * component  (component) object 32B 'Wx' 'Wy' 'Total' 'Speed'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [4]:
pipeline.predict(30)

AttributeError: 'int' object has no attribute 'index'